In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('AAPL', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(-1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 3
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl['3 days future return'].apply(lambda val: to_class(val, 3))
aapl['Return class'].head()

Date
2002-02-11    Neutral
2002-02-12    Neutral
2002-02-13        Neg
2002-02-14        Neg
2002-02-15        Neg
Name: Return class, dtype: object

In [17]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', '3 days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

In [18]:
trafos = [([col], StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
#trafos += [('Return class', LabelBinarizer())]

In [19]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 (['Run Up 252'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Beta 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['EMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA MOM 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Sharpe 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True))]

In [20]:
mapper = DataFrameMapper(trafos, df_out=True)

In [21]:
aapl_mapped = mapper.fit_transform(aapl)
aapl_mapped.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2003-02-11,0.0,0.0,0.0,1.0,0.0,-1.600159,-0.798062,-1.064306,-0.761709,-1.227681,-0.726039,0.437312,-1.151783,-1.292382
2003-02-12,0.0,0.0,0.0,0.0,1.0,-1.578725,-0.915786,-0.977752,-0.604132,0.308279,-0.691778,0.435341,-0.871988,-1.292872
2003-02-13,0.0,0.0,1.0,0.0,0.0,-1.586146,-0.843879,-0.892654,-0.643557,-0.306400,-0.586248,0.423053,-1.027235,-1.293154
2003-02-14,1.0,0.0,0.0,0.0,0.0,-1.558750,-0.818924,-0.809037,-0.604143,-1.536279,-0.517386,0.412020,-0.924379,-1.293432
2003-02-18,0.0,0.0,0.0,1.0,0.0,-1.486968,-0.649921,-0.494489,-0.406986,1.537985,-0.231234,0.418942,-0.965556,-1.293506


# ML model

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split

from sklearn.metrics import accuracy_score, classification_report

In [39]:
X = aapl_mapped
y = aapl['Return class']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [41]:
def fit_and_print(grid):
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    print('Best estimator:\n', str(best_estimator))
    
    y_pred = best_estimator.predict(X_test)
    print('Accuracy score:', str(accuracy_score(y_test, y_pred)))
    print('Classification report\n', classification_report(y_test, y_pred))

### K nearest

In [38]:
params = {'n_neighbors': np.arange(10,100,5),
          'metric': ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock'],
          'algorithm' : ['auto', 'ball_tree', 'kd_tree']}
grid = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4), param_grid=params)

In [ ]:
fit_and_print(grid)

### Random Forest

In [35]:
params = {'n_estimators': np.arange(1, 20, 2),
          'max_features ': np.arange(1, 10, 1),
          'max_depth': np.arange(1, 20, 1),
          'min_samples_split': [2, 3, 4],
          'min_samples_leaf': [1, 2, 3]}
grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=4), param_grid=params)

In [ ]:
fit_and_print(grid)